# BERTopic Pipeline Notebook

This notebook demonstrates how to:

1. **Import** libraries and define the relevant classes/functions for a BERTopic workflow  
2. **Load** documents with metadata, including both posts and comments  
3. **Train** a BERTopic model (using UMAP + HDBSCAN)  
4. **Save** various topic outputs (document info, topic info, and representative documents)  
5. **Generate** HTML visualisations (topics, hierarchy, bar chart)

In [1]:
# ----------------------------------------------------------------------------------------
# 1) Imports and Setup
# ----------------------------------------------------------------------------------------
import json
import os
import traceback
from pathlib import Path

import pandas as pd

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

c:\Users\laure\anaconda3\envs\diss_notebook\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2) Helper Functions

Here we define:
- **`load_docs_with_metadata(file_path)`**: Loads documents (posts + comments) from a JSON structure, returning a combined list of text and relevant metadata.

In [2]:
def load_docs_with_metadata(file_path: Path):
    """
    Loads data from a JSON file (e.g., 'aggregated_raw_reddit_data.json'), 

    Returns a list of (doc_text, metadata_dict) tuples, where doc_text is either
    the post's combined_processed or a comment's comment_processed.
    """
    import json

    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    docs_with_meta = []
    running_index = 0

    for post in data:
        # Basic fields
        post_id = post.get("post_id", "")
        subreddit = post.get("subreddit", "")
        keyword = post.get("keyword", "")
        sub_score = post.get("submission_score", None)

        # Post-level text
        main_text = post.get("combined_processed", "").strip()
        if main_text:
            doc_id_used = post_id if post_id else f"UnknownPost_{running_index}"
            docs_with_meta.append((
                main_text,
                {
                    "doc_idx": running_index,
                    "doc_id": doc_id_used,
                    "subreddit": subreddit,
                    "keyword": keyword,
                    "submission_score": sub_score,
                    "type": "post"
                }
            ))
            running_index += 1

        # Comment-level text
        comments = post.get("comments", [])
        for c in comments:
            c_text = c.get("comment_processed", "").strip()
            if c_text:
                c_id = c.get("comment_id", f"UnknownComment_{running_index}")
                docs_with_meta.append((
                    c_text,
                    {
                        "doc_idx": running_index,
                        "doc_id": c_id,
                        "subreddit": subreddit,
                        "keyword": keyword,
                        "submission_score": sub_score,
                        "type": "comment"
                    }
                ))
                running_index += 1

    return docs_with_meta

## 3) Main Pipeline

This code cell
1. **Loading** the data,
2. **Fitting** a BERTopic model with UMAP & HDBSCAN,
3. **Generating** various data frames of topic and document info,
4. **Saving** JSON files for further analysis,
5. **Attempting** optional HTML visualisations.

In [6]:
def main():
    """
    Main BERTopic pipeline:
    1) Loads 'bertopic_ready_data.json' with custom doc IDs (post_id/comment_id) and text.
    2) Builds a BERTopic model (UMAP + HDBSCAN).
    3) Saves:
       - document-level info (doc_info.json)
       - topic info (topic_info.json).
       - representative docs (topic_representatives.json)
    """
    data_folder = r"C:\Users\laure\Desktop\dissertation_notebook\Data"
    input_path = Path(os.path.join(data_folder, "bertopic_ready_data.json"))
    
    # Output directory
    output_folder = Path(os.path.join(data_folder, "bertopic_output"))
    output_folder.mkdir(parents=True, exist_ok=True)

    if not input_path.exists():
        print(f"File {input_path} not found.")
        return

    # 1) Load docs + metadata
    docs_with_meta = load_docs_with_metadata(input_path)
    docs = [item[0] for item in docs_with_meta]
    metadata_list = [item[1] for item in docs_with_meta]

    #hyperparameters
    n_neighbors = 15
    min_cluster_size = 10

    print("Loading embedding model (CUDA if available)...")
    embedding_model = SentenceTransformer("all-mpnet-base-v2", device="cuda")

    custom_umap = UMAP(
        n_neighbors=n_neighbors,
        n_components=2,
        metric="cosine",
        random_state=42,
        init="random"
    )
    custom_hdbscan = HDBSCAN(
        min_cluster_size=min_cluster_size,
        metric="euclidean",
        cluster_selection_method="eom",
        prediction_data=True
    )

    topic_model = BERTopic(
        embedding_model=embedding_model,
        umap_model=custom_umap,
        hdbscan_model=custom_hdbscan,
        verbose=False
    )

    print(f"Fitting BERTopic with n_neighbors={n_neighbors}, min_cluster_size={min_cluster_size} ...")
    topics, probs = topic_model.fit_transform(docs)
    print("BERTopic modelling complete.")

    # refine displayed topic words using bigrams
    vectorizer = CountVectorizer(stop_words="english", ngram_range=(1, 2))
    topic_model.update_topics(docs, vectorizer_model=vectorizer)

    # 2) Retrieve topic info
    topic_info_df = topic_model.get_topic_info()

    # 3) Document-level info
    doc_info_df = topic_model.get_document_info(docs)
    doc_info_df.rename(columns={"Probability": "Topic_Probability"}, inplace=True)

    # Add custom metadata columns
    doc_info_df["Doc_ID"] = ""
    doc_info_df["Type"] = ""
    doc_info_df["Subreddit"] = ""
    doc_info_df["Keyword"] = ""
    doc_info_df["SubmissionScore"] = None

    for i in range(len(doc_info_df)):
        meta = metadata_list[i]
        doc_info_df.at[i, "Doc_ID"] = meta["doc_id"]
        doc_info_df.at[i, "Type"] = meta["type"]
        doc_info_df.at[i, "Subreddit"] = meta["subreddit"]
        doc_info_df.at[i, "Keyword"] = meta["keyword"]
        doc_info_df.at[i, "SubmissionScore"] = meta["submission_score"]

    # -- REMOVE any fields that store representative docs in doc_info --
    # (This step ensures no snippet arrays or flags like Representative_Docs remain)
    columns_to_drop = ["Representative_Docs", "Representative_document"]
    for col in columns_to_drop:
        if col in doc_info_df.columns:
            doc_info_df.drop(columns=[col], inplace=True)

    # 4) create a separate structure for topic_representatives.json
    representative_docs = []
    valid_topics = sorted(set(doc_info_df["Topic"]) - {-1})
    max_docs_per_id = 2

    for t in valid_topics:
        subset = doc_info_df[doc_info_df["Topic"] == t].copy()
        subset.sort_values("Topic_Probability", ascending=False, inplace=True)

        grouped_by_id = subset.groupby("Doc_ID")
        limited_docs = []
        for doc_id_val, group in grouped_by_id:
            top_docs_per_id = group.head(max_docs_per_id)
            limited_docs.append(top_docs_per_id)

        limited_docs = pd.concat(limited_docs).sort_values("Topic_Probability", ascending=False)
        top10 = limited_docs.head(10)

        # Build rep-doc info with short snippet
        docs_for_topic = []
        for _, row2 in top10.iterrows():
            snippet_text = ""
            if "Document" in row2 and isinstance(row2["Document"], str):
                snippet_text = row2["Document"][:300]
            docs_for_topic.append({
                "Doc_ID": row2["Doc_ID"],
                "Topic_Probability": float(row2["Topic_Probability"]),
                "Type": row2["Type"],
                "Subreddit": row2["Subreddit"],
                "Keyword": row2["Keyword"],
                "SubmissionScore": row2["SubmissionScore"],
                "Snippet": snippet_text
            })

        representative_docs.append({
            "Topic": int(t),
            "NumDocsInTopic": int(subset.shape[0]),
            "TopDocs": docs_for_topic
        })

    # 5) Save outputs
    doc_info_path = output_folder / "doc_info.json"
    topic_info_path = output_folder / "topic_info.json"
    rep_docs_path = output_folder / "topic_representatives.json"

    # doc_info.json: full text, probability, etc. 
    doc_records = doc_info_df.to_dict(orient="records")
    with open(doc_info_path, "w", encoding="utf-8") as f:
        json.dump(doc_records, f, indent=2, ensure_ascii=False)

    # topic_info.json: summary of topics, top words
    topic_records = topic_info_df.to_dict(orient="records")
    with open(topic_info_path, "w", encoding="utf-8") as f:
        json.dump(topic_records, f, indent=2, ensure_ascii=False)

    # topic_representatives.json: separate file with top docs by topic
    with open(rep_docs_path, "w", encoding="utf-8") as f:
        json.dump(representative_docs, f, indent=2, ensure_ascii=False)

    # 6) Generate HTML visuals
    try:
        fig_topics = topic_model.visualize_topics()
        fig_topics.write_html(str(output_folder / "topics.html"))
    except Exception as e:
        print("Could not visualize topics.html")
        traceback.print_exc()

    try:
        fig_hierarchy = topic_model.visualize_hierarchy()
        fig_hierarchy.write_html(str(output_folder / "hierarchy.html"))
    except Exception as e:
        print("Could not visualize hierarchy.html")
        traceback.print_exc()

    # Show largest topics in a bar chart
    topic_info_nonoutlier = topic_info_df[topic_info_df["Topic"] != -1].sort_values("Count", ascending=False)
    top10_topics = topic_info_nonoutlier["Topic"].head(10).tolist()
    try:
        fig_barchart = topic_model.visualize_barchart(topics=top10_topics)
        fig_barchart.write_html(str(output_folder / "barchart.html"))
    except Exception as e:
        print("Could not visualize barchart.html")
        traceback.print_exc()

    print("\nAll done!")
    print(f"Document info saved to: {doc_info_path}")
    print(f"Topic info saved to: {topic_info_path}")
    print(f"Representative docs per topic: {rep_docs_path}")
    print("HTML visualizations in 'bertopic_output' folder.")


if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print("BERTopic pipeline failed.")
        traceback.print_exc()


Loading embedding model (CUDA if available)...
Fitting BERTopic with n_neighbors=15, min_cluster_size=10 ...
BERTopic modelling complete.

All done!
Document info saved to: C:\Users\laure\Desktop\dissertation_notebook\Data\bertopic_output\doc_info.json
Topic info saved to: C:\Users\laure\Desktop\dissertation_notebook\Data\bertopic_output\topic_info.json
Representative docs per topic: C:\Users\laure\Desktop\dissertation_notebook\Data\bertopic_output\topic_representatives.json
HTML visualizations in 'bertopic_output' folder.


## Display resukts

In [22]:
def display_bertopic_results(data_folder):
    """
    Load and display BERTopic results in formatted tables:
    1. An overall topic overview with truncated representative documents
    2. A detailed view of each topic's top documents
    """
    import json
    from IPython.display import display, HTML

    output_folder = Path(data_folder) / "bertopic_output"

    # --- 1) Load results ---
    with open(output_folder / "topic_info.json", 'r', encoding='utf-8') as f:
        topic_info = json.load(f)

    with open(output_folder / "topic_representatives.json", 'r', encoding='utf-8') as f:
        rep_docs = json.load(f)

    # --- 2) Display condensed topic information ---
    topic_df = pd.DataFrame(topic_info)
    topic_df = topic_df[topic_df['Topic'] != -1]  # Excludes the outlier topic
    topic_df = topic_df.sort_values('Count', ascending=False)

    def get_short_rep_examples(topic_id):
        topic_entry = next((d for d in rep_docs if d['Topic'] == topic_id), None)
        if not topic_entry:
            return ""
        topic_docs = topic_entry['TopDocs']
        return '; '.join([d['Snippet'][:100] + '...' for d in topic_docs[:2]])

    topic_df['Representative_Examples'] = topic_df['Topic'].apply(get_short_rep_examples)

    pd.set_option('display.max_colwidth', 200)

    print("\n=== Topic Overview ===")
    display(HTML(topic_df[['Topic', 'Count', 'Name', 'Representative_Examples']].to_html(
        index=False,
        classes='table table-striped'
    )))

    # --- 3) Display detailed topic information ---
    for topic_data in rep_docs:
        topic_id = topic_data['Topic']
        num_docs = topic_data['NumDocsInTopic']

        print(f"\n{'='*80}")
        print(f"\nTopic {topic_id} (Contains {num_docs} documents)")
        print(f"{'-'*80}")

        # Show top 5 representative documents
        docs_data = []
        for doc in topic_data['TopDocs'][:5]:
            docs_data.append({
                'Doc_ID': doc['Doc_ID'],
                'Type': doc['Type'],
                'Subreddit': doc['Subreddit'],
                'Score': doc['SubmissionScore'],
                'Topic Prob': f"{doc['Topic_Probability']:.3f}",
                'Preview': doc['Snippet'][:200] + '...'
            })

        docs_df = pd.DataFrame(docs_data)
        print("\nTop Representative Documents:")
        display(HTML(docs_df.to_html(index=False, classes='table table-striped')))


# Example usage
data_folder = r"C:\Users\laure\Desktop\dissertation_notebook\Data"
display_bertopic_results(data_folder)



=== Topic Overview ===


Topic,Count,Name,Representative_Examples
0,3065,0_diagnosis_autism_people_adhd,"unfortunately just having an afab practitioner wouldn't guarantee your success unless they're active...; i'm audhd so creating and understanding analogies might not be my strong suit, but i'll try. the fir..."
1,1834,1_aba_therapy_child_autistic,aba is basically forcing autistic people to be less autistic and to appear more allistic- which does...; the person who is commenting is coming from a place of knowledge about aba practices of yore. they d...
2,947,2_therapy_therapist_like_just,"i've been to a few different therapists with different methods and it's been unhelpful at best and t...; i actually just broke up with my therapist last night lol. but before her, i was seeing the person w..."
3,62,3_dentist_appointment_dentists_mouth,"my dentist is super careful and gentle while opening my mouth and stuff, she also avoids getting the...; always look up recommendations for gentle densits/hygienists. i will quit a bad one and go to anothe..."
4,57,4_neurodivergent_adhd_people_term,"the unfortunate thing is that the term was coined because people were already using neurodiversity t...; they’re not the most common, dyslexia and dyspraxia are. i have autism and adhd, so there’s nothing ..."
5,42,5_masking_mask_unmasking_just,"there's no treatment in the sense of a cure, but just like there's no treatment for having no legs, ...; i’m 51 am undoubtedly neurodivergent but don’t plan to seek a formal diagnosis. the more i learn abo..."
6,38,6_exposure_exposure therapy_therapy_sensory,"how many of you all were hypersensitive to noises as a kid then had to go to abusive pseudoscience a...; i've heard exposure therapy works for phobias, but not for sensory overload. like some others alread..."
7,31,7_bpd_symptoms_personality disorder_adhd,"i didn't think you thought it wasn't real, just poonting out that this diagnosis is thrown on people...; i wonder if the rampant misdiagnosis of bpd for autistic women by full fledged doctors harms the bpd..."
8,30,8_joke_diagnosis_husband_just,"yeah my sense is, if he's your husband and you've been on this diagnostic journey for a while, he sh...; it could have been a joke that just didn't land. my wife and i have these all the time, and both sus..."
9,25,9_sex_adhd_selfish_husband,this does. not seem like an adhd thing. i'm going to say he's bad at sex. and also kind of an asshol...; being uninterested in pleasing your partner has nothing to do with adhd and i have no idea why you t...




Topic 0 (Contains 3065 documents)
--------------------------------------------------------------------------------

Top Representative Documents:


Doc_ID,Type,Subreddit,Score,Topic Prob,Preview
C_P_100_1792,comment,autism,558,1.000,"unfortunately just having an afab practitioner wouldn't guarantee your success unless they're actively aware of the issue. it's kind of an issue with the whole practice, but definitely still a lot mor..."
C_P_418_8037,comment,neurodiversity,50,1.000,"i'm audhd so creating and understanding analogies might not be my strong suit, but i'll try. the fire analogy doesn't resonate with me, because i don't really think being neurodivergent is painful in ..."
C_P_418_9630,comment,neurodiversity,50,1.000,"having adhd itself doesn't feel like anything. or at least, not that i can tell. that's something that made me doubt it pre-diagnosis. but it's because it's something neurological. i never had a typic..."
C_P_419_4680,comment,AutismInWomen,30,1.000,"i am also in scotland, self referred to the new adult autism service 6 weeks after it launched in 2021, had my assessment earlier this year. so almost 3 years on the waiting list. according to a local..."
C_P_41_1465,comment,autism,684,1.000,"bit late, but i fully agree. i only realized it when i was 35 and first read about aspergers, after reading the symptoms the missing links suddenly appeared. i was tested and diagnosed, and then i rea..."




Topic 1 (Contains 1834 documents)
--------------------------------------------------------------------------------

Top Representative Documents:


...

Doc_ID,Type,Subreddit,Score,Topic Prob,Preview
C_P_439_1509,comment,AutismInWomen,243,1.000,>one of my clients decided to tell me that she thinks vaccines cause autism because of something she read people should just shut up...
C_P_45_7871,comment,autism,421,1.000,"post-diagnosis, my dad told me he’s not a fan of vaccines because he would never want an autistic child...."
C_P_479_1624,comment,autism,155,1.000,didn't the doctor dude who started that whole thing partner up with some wheelchair dude who thought that his own bone marrow would cure it? how do people still believe that vaccines cause autism afte...
C_P_510_3612,comment,AutismInWomen,87,1.000,"because of ex-dr andrew wakefield. he was a british gastroenterologist who - to cut a very, very long story short - developed a competitor to the mmr vaccine. the problem is that his vaccine was exact..."
C_P_532_4917,comment,neurodiversity,103,1.000,...or maybe it’s just a way to spite all the “vaccines cause autism” super moms. that would be pretty funny....




Topic 14 (Contains 13 documents)
--------------------------------------------------------------------------------

Top Representative Documents:


Doc_ID,Type,Subreddit,Score,Topic Prob,Preview
C_P_4_1765,comment,autism,562,1.000,"there’s nothing wrong with humming is there? if he’s loud but that’s his stim, you either have the option of stopping these things or letting them be. it’s not hurting anyone, and if he’s forced to st..."
C_P_4_2499,comment,autism,562,1.000,humming is 100% a healthy and harmless stim. i don’t think there’s any reason to teach your son to suppress it. suppressive a harmless stim can cause more harm than good. i would speak to her or even ...
C_P_4_2637,comment,autism,562,1.000,"i second that! especially as an adult who is still stimming vocally. not because i never got taught how not to, but because sometimes i absolutely have to to deal with a situation at all. there is no ..."
C_P_4_3221,comment,autism,562,1.000,you should be worried because that is gonna cause harm like you said this is the tricky part of aba when they try an take away positive stimming the humming isn't hurting your kiddo or anyone else the...
C_P_4_5766,comment,autism,562,1.000,"aba often just teaches us to mask our autism and being forced to mask led me to depression, panic attacks, meltdowns, and self harm as a teen, and eventual burnout as an adult. any child who is happy ..."




Topic 15 (Contains 12 documents)
--------------------------------------------------------------------------------

Top Representative Documents:


Doc_ID,Type,Subreddit,Score,Topic Prob,Preview
C_P_197_1688,comment,Autism_Parenting,35,1.000,"our son did not have much luck with feeding therapy, but he has expanded his “acceptable” foods a bit with age. at 14 he will eat any processed chicken nugget (not the homemade kind), any brand of box..."
C_P_197_2020,comment,Autism_Parenting,35,1.000,"this was us, we never did food therapy and he actually did start trying a few things when he was almost 6. yes, much of it is carb and sugar based but at least there is variety. we went from 5 foods t..."
C_P_197_307,comment,Autism_Parenting,35,1.000,yes. we do feeding therapy at our aba center. it is slow going and our daughter is still picky but it has expanded her pallet. she will eat two non-hidden veggies now! (cucumbers and orange peppers) t...
C_P_197_3720,comment,Autism_Parenting,35,1.000,yes. we did feeding therapy for a year and it helped tremendously....
C_P_197_6229,comment,Autism_Parenting,35,1.000,"yeah if you want one new food, i think you'll achieve it but it's gonna take some time at the start. that does 100% suck about how much the mcdonalds costs and also, i'd do the same thing you're doing..."


## Conclusion

Load preprocessed data, train a **BERTopic** model, and export multiple outputs:

- **Document Info** – Indicating assigned topics and probabilities for each post/comment, plus metadata such as subreddit and keywords.  
- **Topic Info** – Summaries of each topic, including the number of documents.  
- **Representative Documents** – A sample of the highest probability docs (capped by source ID).  
- **HTML visualisations** (topics, hierarchy, bar chart) in the `bertopic_output` folder.

Refine or extend this process by experimenting with different parameters or exploring the visual outputs. This completes the **BERTopic pipeline** demonstration.

## References

**Reference:**  
Grootendorst, M. (2022) *BERTopic: Leveraging BERT embeddings for unsupervised topic modeling* [computer program].  
Available from: [https://github.com/MaartenGr/BERTopic](https://github.com/MaartenGr/BERTopic) [Accessed 12 January 2025].

**Git Repo:**  
- [BERTopic GitHub](https://github.com/MaartenGr/BERTopic)

**Reference:**  
Reimers, N., and Gurevych, I. (2019) *Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks*.  
Available from: [https://www.sbert.net/](https://www.sbert.net/) [Accessed 12 January 2025].

**Git Repo:**  
- [SentenceTransformers GitHub](https://github.com/UKPLab/sentence-transformers)

**Reference:**  
McInnes, L., Healy, J., and Melville, J. (2018) *UMAP: Uniform Manifold Approximation and Projection for Dimension Reduction* [computer program].  
Available from: [https://umap-learn.readthedocs.io/](https://umap-learn.readthedocs.io/) [Accessed 12 January 2025].

**Git Repo:**  
- [UMAP GitHub](https://github.com/lmcinnes/umap)

**Reference:**  
Campello, R. J. G. B., Moulavi, D., and Sander, J. (2013) *Density-Based Clustering Based on Hierarchical Density Estimates* [computer program].  
Available from: [https://hdbscan.readthedocs.io/](https://hdbscan.readthedocs.io/) [Accessed 12 January 2025].

**Git Repo:**  
- [HDBSCAN GitHub](https://github.com/scikit-learn-contrib/hdbscan)

**Reference:**  
Pedregosa, F., Varoquaux, G., Gramfort, A., Michel, V., Thirion, B., Grisel, O., Blondel, M., et al. (2011) *Scikit-learn: Machine Learning in Python* [software framework]. *Journal of Machine Learning Research*, 12, pp. 2825–2830.  
Available from: [https://scikit-learn.org/](https://scikit-learn.org/) [Accessed 12 January 2025].

**Git Repo:**  
- [Scikit-learn GitHub](https://github.com/scikit-learn/scikit-learn)

**Reference:**  
McKinney, W. (2010) *Data Structures for Statistical Computing in Python*. *Proceedings of the 9th Python in Science Conference*, pp. 51–56 (v2.2.3).  
Available from: [https://pandas.pydata.org/](https://pandas.pydata.org/) [Accessed 12 January 2025].

**Git Repo:**  
- [Pandas GitHub](https://github.com/pandas-dev/pandas)